In [17]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import csv
from collections import defaultdict

In [18]:
def split_into_size_groups(data: pd.DataFrame):
    # This function splits up the dataframe into 3 corresponding to the sizes of 25x4, 10x10, and 4x25 for analysis by group

    # Contexts' 0-24 are the first 25x4, contexts' 25-34 are the first 10x10, contexts' 35-38 are the first 4x25. then the context value resets
    #   to 0 and this continues
    df_list = []
    size_groups = [(0, 25), (25, 35), (35, 39)]
    for start, end in size_groups:
        df_group = data[(data['context'] >= start) & (data['context'] < end)].reset_index(drop=True)
        df_list.append(df_group)

    return df_list

In [19]:
def unpack_data(filename: str):
    df = pd.read_csv(filename)
    return df

In [20]:
print("DEBUG: Unpacking data")
plot_savename = "/home/ia23938/Documents/GitHub/ValueSystemsAggregation/bluepebble_plots/"
results_path = "/home/ia23938/Documents/GitHub/ValueSystemsAggregation/bluepebble_runs/experiment_results_2024-09-27/"
results_filename = {'egal': "egal_society/egal_societyegal_society.csv", 'norm': "norm_society/norm_societynorm_society.csv", "util": "util_society/util_societyutil_society.csv", "random": "rand_society/rand_societyrand_society.csv"}

data = unpack_data(results_path + results_filename["util"])

#data = split_into_size_groups(data)

#data[0]

#for name, filename in results_filename.items():
#    data = unpack_data(results_path + filename)


DEBUG: Unpacking data


In [25]:
worst_agent = 101

df_dict = {}
unique_agent_values = data['agent'].unique()
# Split the DataFrame by P_Value
for agent in unique_agent_values:
    df_dict[f'df_p_{agent}'] = data[data['agent'] == agent].reset_index(drop=True)
for key in df_dict.keys():
    df_dict[key] = df_dict[key].sort_values(by=['agent', 'context']).reset_index(drop=True)
for key in df_dict:
    # Calculate cumulative sum of Satisfaction for each Agent
    df_dict[key]['Cumulative_Satisfaction'] = df_dict[key].groupby('agent')['satisfaction'].cumsum()
for key in df_dict:
    # Reset the Context for each Agent group
    df_dict[key]['context'] = df_dict[key].groupby('agent').cumcount()

print(unique_agent_values[0])
print("df_dict", df_dict.keys())
df_dict['df_p_41.0']

78.0
df_dict dict_keys(['df_p_78.0', 'df_p_69.0', 'df_p_68.0', 'df_p_25.0', 'df_p_41.0', 'df_p_10.0', 'df_p_95.0', 'df_p_56.0', 'df_p_96.0', 'df_p_83.0', 'df_p_54.0', 'df_p_90.0', 'df_p_80.0', 'df_p_64.0', 'df_p_45.0', 'df_p_82.0', 'df_p_49.0', 'df_p_34.0', 'df_p_88.0', 'df_p_98.0', 'df_p_37.0', 'df_p_48.0', 'df_p_17.0', 'df_p_57.0', 'df_p_42.0', 'df_p_4.0', 'df_p_50.0', 'df_p_26.0', 'df_p_67.0', 'df_p_52.0', 'df_p_70.0', 'df_p_22.0', 'df_p_74.0', 'df_p_18.0', 'df_p_58.0', 'df_p_3.0', 'df_p_92.0', 'df_p_33.0', 'df_p_77.0', 'df_p_7.0', 'df_p_1.0', 'df_p_76.0', 'df_p_43.0', 'df_p_8.0', 'df_p_94.0', 'df_p_21.0', 'df_p_16.0', 'df_p_19.0', 'df_p_46.0', 'df_p_55.0', 'df_p_9.0', 'df_p_87.0', 'df_p_14.0', 'df_p_0.0', 'df_p_40.0', 'df_p_23.0', 'df_p_60.0', 'df_p_44.0', 'df_p_71.0', 'df_p_39.0', 'df_p_62.0', 'df_p_30.0', 'df_p_99.0', 'df_p_32.0', 'df_p_72.0', 'df_p_63.0', 'df_p_13.0', 'df_p_6.0', 'df_p_85.0', 'df_p_24.0', 'df_p_27.0', 'df_p_2.0', 'df_p_81.0', 'df_p_65.0', 'df_p_36.0', 'df_p_29.0

,context,p_value,agent,satisfaction,Cumulative_Satisfaction
0,0,0,41.0,0.185803,0.185803
1,1,1,41.0,0.019978,0.205781
2,2,2,41.0,0.060468,0.266250
3,3,3,41.0,0.186970,0.453219
4,4,0,41.0,0.356364,0.809584
...,...,...,...,...,...
1291,1291,3,41.0,0.547334,280.875817
1292,1292,0,41.0,0.007929,280.883746
1293,1293,1,41.0,0.007364,280.891110
1294,1294,2,41.0,0.028831,280.919941
